In [1]:
"""
Purpose: To Inspect the Autoproofreading Pipeline
and the resultant connectome

"""

'\nPurpose: To Inspect the Autoproofreading Pipeline\nand the resultant connectome\n\n'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from os import sys
sys.path.append("/meshAfterParty/")

In [4]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-02-08 07:11:48,390 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-02-08 07:11:48,391 - settings - Setting database.user to celiib
INFO - 2021-02-08 07:11:48,392 - settings - Setting database.password to newceliipass
INFO - 2021-02-08 07:11:48,397 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-02-08 07:11:48,398 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-08 07:11:48,410 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-02-08 07:11:48,693 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-08 07:11:48,725 - settings - Setting enable_python_native_blobs to True
INFO - 2021-02-08 07:11:49,026 - settings - Setting enable_python_native_blobs to True


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import networkx_utils as xu

In [6]:
len(minnie.AutoProofreadNeurons())

76928

# Getting the Errors

In [ ]:
"""
Notes on Error: the decompression error was usually from SIGTERM signal


"""

In [7]:
err_table = (schema.jobs & "table_name='__auto_proofread_neurons'")
err_messages = (err_table).fetch("error_message")
unique_errors,unique_counts = np.unique(err_messages,return_counts=True)
high_frequency_errors = np.where(unique_counts>20)[0]
unique_errors[high_frequency_errors]

array(['', 'DataJointError: Attempt to insert rows with different fields',
       'Exception: Number of Error Limbs was greater than 1: 1',
       'Exception: Number of Somas was greater than 1: 2',
       'Exception: not downsampled branch',
       'IndexError: index 1 is out of bounds for axis 0 with size 1',
       'IndexError: index 2 is out of bounds for axis 0 with size 1',
       'IndexError: index 2 is out of bounds for axis 0 with size 2',
       "KeyError: 'skeletal_length'", 'SystemExit: SIGTERM received'],
      dtype=object)

In [38]:
new_table = du.restrict_jobs_table_by_error_substring(err_table,
                                         message_in_error="skeletal_length")
new_table

Found 0 matching entries with substring skeletal_length


table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp


In [39]:
new_table.delete()

In [26]:
specific_error = (err_table & "key_hash='0171028a1fd889dec937f482e1e88b28'")
specific_error

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__auto_proofread_neurons,0171028a1fd889dec937f482e1e88b28,error,=BLOB=,KeyError: 'skeletal_length',=BLOB=,celiib@10.66.4.64,jr-compute001,1,118581,2021-02-05 00:31:27


In [27]:
print(specific_error.fetch1("error_stack"))

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/indexes/base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'skeletal_length'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "Run_9_AutoProofreadNeurons.py", line 408, in make
    verbose=False)
  File "/meshAfterParty/proofreading_utils.py", line 3591, in proofread_neuron
    verbose=verbose)
  Fi

In [28]:
print(specific_error.fetch1("key"))

{'segment_id': 864691135651861970, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 5, 'index': 0}


In [29]:
specific_error.delete()

In [ ]:
minnie.SpineRecalculation() & dict(segment_id=864691135864722652)

# Getting the Connectome

In [ ]:
minnie.ProofreadStats() & (minnie.AutoProofreadNeurons() & "cell_type_predicted='inhibitory'").proj() & "axon_on_dendrite_merges_error_area>=0"

In [ ]:
minnie.SynapseProofread()

In [ ]:
# Fetching all of the data

In [ ]:
synapse_data = minnie.SynapseProofread.fetch(as_dict=True)
synapse_data_df = pd.DataFrame.from_dict(synapse_data)
synapse_data_df

In [ ]:
uniq_synapse_ids,unique_synapse_indexes,inverse_indices,synapse_id_counts = np.unique(synapse_data_df["synapse_id"].to_numpy(),
                           return_index=True,
                           return_inverse=True,
                            return_counts=True)

In [ ]:
direct_connection_synapse_idx = np.where(synapse_id_counts==2)[0]
direct_connection_synapse_mask = np.zeros(len(synapse_id_counts))
direct_connection_synapse_mask[direct_connection_synapse_idx] = 1
direct_connection_synapse_mask = direct_connection_synapse_mask.astype("bool")
row_filter = direct_connection_synapse_mask[inverse_indices]

direct_connections_df = synapse_data_df.iloc[row_filter]
direct_connections_df

In [ ]:
# Combining the table's presyns and postsyns onto the same row

df_tables = []
for synapse_type in ["presyn","postsyn"]:
    direct_connections_df_init = direct_connections_df[direct_connections_df["synapse_type"]==synapse_type]
    
    direct_connections_df_init_sorted = direct_connections_df_init.sort_values(by=['synapse_id']).reset_index(drop=True)
    
    direct_connections_df_presyn = direct_connections_df_init_sorted[[k for k in direct_connections_df_init_sorted.columns if k != "synapse_type"] ]
    
    #column_mapper = dict([(k,f"{k}_{synapse_type}") if k != "synapse_id"  else (k,k) for k in direct_connections_df_presyn.columns])
    column_mapper = dict([(k,f"{synapse_type}_{k}") for k in direct_connections_df_presyn.columns])

    presyn_df = direct_connections_df_presyn.rename(columns=column_mapper)
    df_tables.append(presyn_df)
    
direct_conn_df = pd.concat(df_tables,axis=1)
direct_conn_df

In [ ]:
edges_df = direct_conn_df[["presyn_nucleus_id","postsyn_nucleus_id"]]
connectome_edges = edges_df.to_numpy()
connectome_edges

# Preliminary Look at the Graph

In [ ]:
import networkx as nx

In [ ]:
G = nx.MultiDiGraph()
G.add_edges_from(connectome_edges)
len(G.nodes()),len(G.edges())

In [ ]:
#For connected components cannot be a directed graph
G_undirected = nx.Graph(G)

n_components = nx.number_connected_components(G_undirected)
print(f"Number of connected components = {n_components}")

# The Sizes of the components in the graph
conn_comp = list(nx.connected_components(G_undirected))
conn_comp_size = [len(k) for k in conn_comp]
print(f"conn_comp_size = {conn_comp_size}")

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(conn_comp_size)
ax.set_title("Number of Neurons in Connected Components")
ax.set_xlabel("Number of Neurons")
plt.show()

# Selecting the largest component

In [ ]:
G_largest = G.subgraph(conn_comp[0])
node_names_largest_comp = np.array(G_largest.nodes())
node_names_largest_comp.shape

# Degree Distribution

In [ ]:
undirectional_graph = nx.Graph(G_largest)
no_selflooped_graph = xu.remove_selfloops(undirectional_graph)
no_selflooped_graph

In [ ]:
degree_distribution = np.array(xu.get_node_degree(no_selflooped_graph,node_names_largest_comp))
degree_distribution_filtered = degree_distribution[degree_distribution<np.percentile(degree_distribution,99.5)]

In [ ]:
print(f"Average Degree of Node: {np.mean(degree_distribution_filtered)}")

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(degree_distribution_filtered,bins=50,density=True)
ax.set_title("Degree Distribution of Neurons \n Unique Edges, No Self Loops")
ax.set_xlabel("Degree of Neuron")
ax.set_ylabel("Density")
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(degree_distribution_filtered,bins=50,density=False)
ax.set_title("Degree Distribution of Neurons \n Unique Edges, No Self Loops")
ax.set_xlabel("Degree of Neuron")
ax.set_ylabel("Count")
ax.set_yscale("log")
plt.show()

# Further Graph Analysis

In [ ]:
#this can take a little bit to compute
nx.average_shortest_path_length(G_largest)

# OLD Datajoint Way of Processing

In [ ]:
""" OLD Datajoint Way

presyn_table = minnie.SynapseProofread & "synapse_type='presyn'"
postsyn_table = minnie.SynapseProofread & "synapse_type='postsyn'"
presyn_table_renamed = presyn_table.proj(presyn="segment_id",
                                         presyn_split_index = "split_index",
                                         presyn_nucleus="nucleus_id",
                                         presyn_label="synapse_type")

postsyn_table_renamed = postsyn_table.proj(postsyn="segment_id",
                                           postsyn_split_index="split_index",
                                            postsyn_nucleus="nucleus_id",
                                           postsyn_label="synapse_type")

direct_conn_with_labels = presyn_table_renamed * postsyn_table_renamed
direct_conn_with_labels

import time
st_time = time.time()
direct_conn_data_without_sizes = direct_conn_with_labels.fetch(as_dict=True)
print(f"Total time for fetch = {time.time() - st_time}")

import datajoint as dj
just_synapses_id_and_size = dj.U("synapse_id","synapse_size") & du.m65mat.NucleusSynapse.proj("synapse_size")

# want to get the synapse table as well
direct_conn_with_labels_and_sizes = just_synapses_id_and_size * direct_conn_with_labels
direct_conn_with_labels_and_sizes

import time
st_time = time.time()
direct_conn_data = direct_conn_with_labels_and_sizes.fetch(as_dict=True)
print(f"Total time for fetch = {time.time() - st_time}")

import pandas as pd
direct_conn_df = pd.DataFrame.from_dict(direct_conn_data)
"""

In [ ]:
(1/3)**-2